## Day 16: Packet Decoder

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day16/puzzle.ipynb)

[Problem statement](https://adventofcode.com/2021/day/16)

In [ ]:
#load "../common.fsx"

let toInt s = Convert.ToInt64(s, 2)
let charsToInt chars = chars |> Seq.toArray |> String |> toInt

[<AutoOpen>]
module rec Types = 
    type Packet = { Version: int; Body: PacketBody }
    type Operation = 
        | Sum
        | Product
        | Min
        | Max
        | Gt
        | Lt
        | Eq
    type Operator = { Operation: Operation; Nested: Packet list }
    type PacketBody = 
        | Literal of Value: int64
        | Operator of Operator


In [ ]:
module rec Parsing = 
    let private cutPrefix count (input: char list) = 
        let (prefixPart, rest) = List.splitAt count input
        charsToInt prefixPart |> int, rest

    let private handleLiteral (input: char list) =
        let blocks =
            input
            |> Seq.chunkBySize 5
            |> Seq.takeUntil (fun chars -> chars.[0] ='0')
            |> Seq.toArray
        let value = 
            blocks
            |> Seq.map (fun chars -> chars.[1..] |> String)
            |> String.concat ""
            |> toInt

        Literal value, input |> List.skip ((blocks |> Array.length) * 5)

    let private handlePaketSet (input: char list) =
        match input with
        | '0'::restWoSetType ->
            let (charCount, restWoCount) = cutPrefix 15 restWoSetType
            let (thisSetPart, continuation) = List.splitAt charCount restWoCount
            let packets = List.unfold (function | [] -> None | rest -> Some (handlePacket rest)) thisSetPart
            packets, continuation
        | '1'::restWoSetType ->
            let (packetCount, restWoCount) = cutPrefix 11 restWoSetType
            [1..packetCount]
            |> List.mapFold (fun rest _ -> handlePacket rest) restWoCount
    
    let private handleVersion (input: char list) =
        cutPrefix 3 input

    let private handlePacket (input: char list) =
        let (version, bodyInput) = handleVersion input
        let (body, rest) = handleBody bodyInput
        { Version = version; Body = body }, rest

    let private handleBody (input: char list) =
        let (``type``, rest) = cutPrefix 3 input
        match ``type`` with
        | 4 -> handleLiteral rest
        | opType -> 
            let (nested, continuation) = handlePaketSet rest
            match opType with
            | 0 -> Operator { Operation = Sum; Nested = nested }, continuation
            | 1 -> Operator { Operation = Product; Nested = nested }, continuation
            | 2 -> Operator { Operation = Min; Nested = nested }, continuation
            | 3 -> Operator { Operation = Max; Nested = nested }, continuation
            | 5 -> Operator { Operation = Gt; Nested = nested }, continuation
            | 6 -> Operator { Operation = Lt; Nested = nested }, continuation
            | 7 -> Operator { Operation = Eq; Nested = nested }, continuation


    let private hexToBinList (hexString: string) = 
        hexString 
        |> Seq.map (fun c -> Convert.ToInt32(string c, 16))
        |> Seq.collect(fun h -> Convert.ToString(h, 2).PadLeft(4, '0'))
        |> List.ofSeq

    let parseFromHex hexString =
        hexString 
        |> hexToBinList
        |> handlePacket
        |> fst


In [ ]:
let versionSum (packet : Packet) = 
    let rec sumRec (acc: int) (packet : Packet) : int= 
        match packet with
        | { Version = version; Body = (Literal _) } -> acc + version
        | { Version = version; Body = (Operator x) } -> version + List.fold sumRec acc x.Nested 
    sumRec 0 packet
    

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
Parsing.parseFromHex "8A004A801A8002F478" |> versionSum |> display
Parsing.parseFromHex "620080001611562C8802118E34" |> versionSum |> display
Parsing.parseFromHex "C0015000016115A2E0802F182340" |> versionSum |> display
Parsing.parseFromHex "A0016C880162017C3686B18A3D4780" |> versionSum |> display

16

12

23

31

In [ ]:
#!share inputRaw --from value
Parsing.parseFromHex inputRaw |> versionSum |> display


955

In [ ]:
let rec calculate (packet : Packet) = 
    match packet.Body with
    | Literal value -> value
    | Operator x -> 
        let apply f packets = 
            packets
            |> List.map calculate
            |> List.reduce f
        let {Operation = operation; Nested = nested} = x
        match operation with
        | Sum -> apply (+) nested
        | Product -> apply (*) nested
        | Min -> apply (min) nested
        | Max -> apply (max) nested
        | Gt -> apply (fun x1 x2 -> if x1 > x2 then 1L else 0L) nested
        | Lt -> apply (fun x1 x2 -> if x1 < x2 then 1L else 0L) nested
        | Eq -> apply (fun x1 x2 -> if x1 = x2 then 1L else 0L) nested

In [ ]:
Parsing.parseFromHex "C200B40A82" |> calculate |> display
Parsing.parseFromHex "04005AC33890" |> calculate |> display
Parsing.parseFromHex "880086C3E88112" |> calculate |> display
Parsing.parseFromHex "CE00C43D881120" |> calculate |> display
Parsing.parseFromHex "D8005AC2A8F0" |> calculate |> display
Parsing.parseFromHex "F600BC2D8F" |> calculate |> display
Parsing.parseFromHex "9C005AC2F8F0" |> calculate |> display
Parsing.parseFromHex "9C0141080250320F1802104A08" |> calculate |> display


3

54

7

9

1

0

0

1

In [ ]:
Parsing.parseFromHex inputRaw |> calculate |> display

158135423448